In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
import numpy as np
import pandas as pd

In [2]:
class MysteryImageDataset(Dataset):
    def __init__(self, csv_file, is_test=False):
        self.is_test = is_test
        df = pd.read_csv(csv_file)
        
        self.ids = df.iloc[:, 0].values
        
        if self.is_test:
            self.X = df.iloc[:, 1:].values.astype(np.float32)
            self.y = None
        else:
            self.X = df.iloc[:, 1:-1].values.astype(np.float32)
            self.y = df.iloc[:, -1].values.astype(np.int64) 

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        features_1d = self.X[idx] # Shape (205,)
        
        features_padded = np.pad(features_1d, (0, 20), 'constant', constant_values=0)
        
        
        image = torch.tensor(features_padded).view(1, 15, 15)
        
        if self.is_test:
            return image
        else:
            label = torch.tensor(self.y[idx])
            return image, label

In [3]:
# import data from files
full_dataset = MysteryImageDataset("data/train.csv", is_test=False)
submission_dataset = MysteryImageDataset("data/test.csv", is_test=True)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_data, val_data = random_split(full_dataset, [train_size, val_size])


In [4]:
# create data loaders
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(submission_dataset, batch_size=batch_size, shuffle=False) 

In [5]:
input_dim = 205

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2), 

            nn.Conv2d(16, 32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2) 
            
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(64, 512), 
            nn.ReLU(),
            nn.Linear(512, 256), 
            nn.ReLU(),
            nn.Linear(256, 5) # 5 Classes
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

In [7]:
device = torch.cuda.current_device().type if torch.cuda.is_available() else "cpu"
model = SimpleCNN().to(device)
learning_rate = 1e-3    
batch_size = 64
epochs = 5

In [ ]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
    return running_loss / len(dataloader.dataset)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(logits, 1)
            correct += (preds == labels).sum().item()
    return running_loss / len(dataloader.dataset), correct / len(dataloader.dataset)

def test_loop(dataloader, model, output_file):
    model.eval()
    all_preds = []
    
    
    
    with torch.no_grad():
        # Note: The test_dataloader only returns 'images', not (images, labels)
        for images in dataloader:
            images = images.to(device)
            
            
            logits = model(images)
            
            
            batch_preds = logits.argmax(1).cpu().numpy()
            all_preds.extend(batch_preds)
            
    submission_df = pd.DataFrame({
        "id": dataloader.dataset.ids, 
        "label": all_preds
    })
    

    submission_df.to_csv(output_file, index=False)
    print(f"Done! Saved {len(submission_df)} predictions to '{output_file}'")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

num_epochs = 15
print("Starting Training...")
for epoch in range(num_epochs):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device)
    val_loss, val_acc = evaluate(model, val_dataloader, loss_fn, device)
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")
print("Done!")




Starting Training...
Epoch 1/15 | Train Loss: 1.2369 | Val Loss: 1.0980 | Val Acc: 59.44%
Epoch 2/15 | Train Loss: 0.9608 | Val Loss: 0.9752 | Val Acc: 64.94%
Epoch 3/15 | Train Loss: 0.7993 | Val Loss: 0.9547 | Val Acc: 67.56%
Epoch 4/15 | Train Loss: 0.6966 | Val Loss: 0.9328 | Val Acc: 66.69%
Epoch 5/15 | Train Loss: 0.6130 | Val Loss: 0.8569 | Val Acc: 69.06%
Epoch 6/15 | Train Loss: 0.5262 | Val Loss: 0.8481 | Val Acc: 70.12%
Epoch 7/15 | Train Loss: 0.4720 | Val Loss: 0.8001 | Val Acc: 71.12%
Epoch 8/15 | Train Loss: 0.4051 | Val Loss: 0.8832 | Val Acc: 70.25%
Epoch 9/15 | Train Loss: 0.3844 | Val Loss: 0.8407 | Val Acc: 72.75%
Epoch 10/15 | Train Loss: 0.3287 | Val Loss: 0.8603 | Val Acc: 73.44%
Epoch 11/15 | Train Loss: 0.2724 | Val Loss: 0.8412 | Val Acc: 73.75%
Epoch 12/15 | Train Loss: 0.2671 | Val Loss: 0.8821 | Val Acc: 70.25%
Epoch 13/15 | Train Loss: 0.2163 | Val Loss: 1.1131 | Val Acc: 73.81%
Epoch 14/15 | Train Loss: 0.1775 | Val Loss: 1.0456 | Val Acc: 72.38%
Epoch 15

In [13]:
test_loop(test_dataloader, model, "submission4.csv")

AttributeError: 'DataLoader' object has no attribute 'eval'